# Interactive script - Performance of the wind turbine model IEA 3.4 MW subjected to inflow misalignment and operated with a variable speed, collective pitch style controller.

 This is an interactive version of Figure 6 found in Tamaro et al (in review). If you are not interested in the code, you may skip this first section and simply adjust the parameters in the [Variables](#set_vars) section to your needs. Once you are done, rerun the section to generate the new plot.

## Code

In [ ]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import fsolve
from scipy.interpolate import RegularGridInterpolator
import matplotlib.pyplot as plt

 Define function to compute $C_T$

In [ ]:
def find_ct(x,*data):
    sigma,cd,cl_alfa,gamma,delta,k,cosMu,sinMu,tsr,theta,mu = data
    CD = np.cos(np.deg2rad(delta))
    CG = np.cos(np.deg2rad(gamma))
    SD = np.sin(np.deg2rad(delta))
    SG = np.sin(np.deg2rad(gamma))
    a = (1- ( (1+np.sqrt(1-x-1/16*x**2*sinMu**2))/(2*(1+1/16*x*sinMu**2))) )    
    k_1s = -1*(15*np.pi/32*np.tan((mu+np.sin(mu)*(x/2))/2));
    I1 = -(np.pi*cosMu*(tsr - CD*SG*k)*(a - 1) 
           + (CD*CG*cosMu*k_1s*SD*a*k*np.pi*(2*tsr - CD*SG*k))/(8*sinMu))/(2*np.pi)
    I2 = (np.pi*sinMu**2 + (np.pi*(CD**2*CG**2*SD**2*k**2 
                                   + 3*CD**2*SG**2*k**2 - 8*CD*tsr*SG*k 
                                   + 8*tsr**2))/12)/(2*np.pi)
    return (sigma*(cd+cl_alfa)*(I1) - sigma*cl_alfa*theta*(I2)) - x

 Define function to compute $C_P$

In [ ]:
def find_cp(sigma,cd,cl_alfa,gamma,delta,k,cosMu,sinMu,tsr,theta,ct,mu):
    a = 1-((1+np.sqrt(1-ct-1/16*sinMu**2*ct**2))/(2*(1+1/16*ct*sinMu**2)))
    SG = np.sin(np.deg2rad(gamma))
    CG = np.cos(np.deg2rad(gamma))                
    SD = np.sin(np.deg2rad(delta))  
    CD = np.cos(np.deg2rad(delta))  
    k_1s = -1*(15*np.pi/32*np.tan((mu+np.sin(mu)*(ct/2))/2));
    #
    cp = sigma*((np.pi*cosMu**2*tsr*cl_alfa*(a - 1)**2 
                 - (tsr*cd*np.pi*(CD**2*CG**2*SD**2*k**2 + 3*CD**2*SG**2*k**2 - 8*CD*tsr*SG*k + 8*tsr**2))/16 
                 - (np.pi*tsr*sinMu**2*cd)/2 - (2*np.pi*cosMu*tsr**2*cl_alfa*theta)/3 
                 + (np.pi*cosMu**2*k_1s**2*tsr*a**2*cl_alfa)/4 
                 + (2*np.pi*cosMu*tsr**2*a*cl_alfa*theta)/3 + (2*np.pi*CD*cosMu*tsr*SG*cl_alfa*k*theta)/3 
                 + (CD**2*cosMu**2*tsr*cl_alfa*k**2*np.pi*(a - 1)**2*(CG**2*SD**2 + SG**2))/(4*sinMu**2) 
                 - (2*np.pi*CD*cosMu*tsr*SG*a*cl_alfa*k*theta)/3 
                 + (CD**2*cosMu**2*k_1s**2*tsr*a**2*cl_alfa*k**2*np.pi*(3*CG**2*SD**2 + SG**2))/(24*sinMu**2) 
                 - (np.pi*CD*CG*cosMu**2*k_1s*tsr*SD*a*cl_alfa*k)/sinMu 
                 + (np.pi*CD*CG*cosMu**2*k_1s*tsr*SD*a**2*cl_alfa*k)/sinMu 
                 + (np.pi*CD*CG*cosMu*k_1s*tsr**2*SD*a*cl_alfa*k*theta)/(5*sinMu) 
                 - (np.pi*CD**2*CG*cosMu*k_1s*tsr*SD*SG*a*cl_alfa*k**2*theta)/(10*sinMu))/(2*np.pi))
    
    return cp

 Define function to get tip speed ratio

In [ ]:
def get_tsr(x,*data):
    yaw,tilt,u,pitch_in,omega_lut_pow,torque_lut_omega,shear,rho,cd,cl_alfa,beta = data
    cp_table_iea    = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/Cp_335.mat')['num']);
    pitch_table_iea = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/pitch_335.mat')['num']);
    tsr_table_iea   = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/TSR_335.mat')['num']);
    u_table_iea     = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/U_335.mat')['num']);
    #
    omega_lut_torque = omega_lut_pow*np.pi/30;
    sigma = 0.0416
    R       = 65;
    omega   = x*u/R;
    omega_rpm = omega*30/np.pi;
    #
    pitch_in = pitch_in;
    pitch_deg = pitch_in;
    #
    torque_nm = np.interp(omega,omega_lut_torque,torque_lut_omega);
    #
    mu    = np.arccos(np.cos(np.deg2rad(gamma))*np.cos(np.deg2rad(tilt)))
    data  = (sigma,cd,cl_alfa,gamma,tilt,shear,np.cos(mu),np.sin(mu),x,np.deg2rad(pitch_in)+np.deg2rad(beta),mu)
    x0    = 0.8
    ct = fsolve(find_ct, x0,args=data)[0]
    cp = find_cp(sigma,cd,cl_alfa,gamma,tilt,shear,np.cos(mu),np.sin(mu),x,np.deg2rad(pitch_in)+np.deg2rad(beta),ct,mu)
    #
    mu    = np.arccos(np.cos(np.deg2rad(0))*np.cos(np.deg2rad(tilt)))
    data  = (sigma,cd,cl_alfa,0,tilt,shear,np.cos(mu),np.sin(mu),x,np.deg2rad(pitch_in)+np.deg2rad(beta),mu)
    x0    = 0.8
    ct = fsolve(find_ct, x0,args=data)[0]
    cp0 = find_cp(sigma,cd,cl_alfa,0,tilt,shear,np.cos(mu),np.sin(mu),x,np.deg2rad(pitch_in)+np.deg2rad(beta),ct,mu)
    #
    eta_p = cp/cp0;
    interp   = RegularGridInterpolator((np.squeeze((u_table_iea)),np.squeeze((tsr_table_iea)),
                        np.squeeze((pitch_table_iea))), cp_table_iea,
                                       bounds_error=False, fill_value=None)
    #
    Cp_now = interp((9,x,pitch_deg),method='linear');
    cp_g1 =  Cp_now*eta_p;
    #
    aero_pow = 0.5*rho*(np.pi*R**2)*(0.975*u)**3*cp_g1;
    electric_pow = torque_nm*(omega_rpm*np.pi/30);
    #
    y = aero_pow - electric_pow
    return y

 Define function to get pitch angle

In [ ]:
def get_pitch(x,*data):
    gamma,tilt,u,omega_rated,omega_lut_torque,torque_lut_omega,shear,rho,cd,cl_alfa,beta = data
    cp_table_iea    = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/Cp_335.mat')['num']);
    pitch_table_iea = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/pitch_335.mat')['num']);
    tsr_table_iea   = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/TSR_335.mat')['num']);
    u_table_iea     = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/U_335.mat')['num']);
    #    
    sigma = 0.0416
    #
    R       = 65;
    omega_rpm   = omega_rated*30/np.pi;
    tsr     = omega_rated*R/(u);
    #
    pitch_in = np.deg2rad(x);
    torque_nm = np.interp(omega_rpm,omega_lut_torque*30/np.pi,torque_lut_omega);
    #   
    mu    = np.arccos(np.cos(np.deg2rad(gamma))*np.cos(np.deg2rad(tilt)))
    data  = (sigma,cd,cl_alfa,gamma,tilt,shear,np.cos(mu),np.sin(mu),tsr,(pitch_in)+np.deg2rad(beta),mu)
    x0    = 0.8
    ct = fsolve(find_ct, x0,args=data)[0]
    cp = find_cp(sigma,cd,cl_alfa,gamma,tilt,shear,np.cos(mu),np.sin(mu),tsr,(pitch_in)+np.deg2rad(beta),ct,mu)
    #
    mu    = np.arccos(np.cos(np.deg2rad(0))*np.cos(np.deg2rad(tilt)))
    data  = (sigma,cd,cl_alfa,0,tilt,shear,np.cos(mu),np.sin(mu),tsr,(pitch_in)+np.deg2rad(beta),mu)
    x0    = 0.8
    ct = fsolve(find_ct, x0,args=data)[0]
    cp0 = find_cp(sigma,cd,cl_alfa,0,tilt,shear,np.cos(mu),np.sin(mu),tsr,(pitch_in)+np.deg2rad(beta),ct,mu)
    #
    eta_p = cp/cp0;
    interp   = RegularGridInterpolator((np.squeeze((u_table_iea)),np.squeeze((tsr_table_iea)),
                        np.squeeze((pitch_table_iea))), cp_table_iea,
                                       bounds_error=False, fill_value=None)
    #
    Cp_now = interp((9,tsr,x),method='linear');
    cp_g1 =  Cp_now*eta_p;
    aero_pow = 0.5*rho*(np.pi*R**2)*(0.975*u)**3*cp_g1;
    electric_pow = torque_nm*(omega_rpm*np.pi/30);
    #
    y = aero_pow - electric_pow
    return y

## Variables<br>
Define variables<br>
You may set the variables and rerun the cells below by selecting run in the menu above.

In [ ]:
u_v      = 10.5;       # free-stream wind speed
shear     = 0.2;    # linear shear
rho       = 1.22;   # air density
tilt      = -5;    # rotor tilt angle [deg]

 Wind turbine and model parameters

In [ ]:
cd              = 0.0051                # model drag coefficient      [-]
cl_alfa         = 4.7662                # model lift slope            [1/rad]
beta            = -3.336                # model blade twist angle     [deg]
R         = 65;     # Rotor radius
sigma     = 0.0416; # Rotor solidity
A         = np.pi*R**2 # rotor area [m2]
omega_cut_in = 2     # RPM
omega_max    = 11.75 # RPM
rated_power_aero  = 3.66e6  # MW

 Load turbine performance curves

In [ ]:
cp_table_iea    = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/Cp_335.mat')['num']);
ct_table_iea    = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/Ct_335.mat')['num']);
pitch_table_iea = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/pitch_335.mat')['num']);
tsr_table_iea   = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/TSR_335.mat')['num']);
u_table_iea     = np.squeeze(loadmat('Cp-Lambda curves 335MW_2/U_335.mat')['num']);

 Get optimum operation point, first crop the table of Cp ...

In [ ]:
pitch_low = -2
pitch_high = 25
tsr_low = 3
tsr_high = 10
u_target = 9.75
idxP = np.squeeze(np.where((pitch_table_iea>pitch_low) & (pitch_table_iea<pitch_high) ))
idxT = np.squeeze(np.where((tsr_table_iea>tsr_low) & (tsr_table_iea<tsr_high) ))
idxU = np.argmin(np.abs(u_table_iea-u_target))
idx = np.squeeze(np.where(cp_table_iea == np.max(cp_table_iea[idxU,idxT[0]:idxT[-1],idxP[0]:idxP[-1]])))
pitch_opt = pitch_table_iea[idx[2]]
tsr_opt   = 8
max_cp    = cp_table_iea[idx[0],20,idx[2]]
## Compute torque-rpm relation and check for region 2-and-a-half
Region2andAhalf = False
omega_array = np.linspace(omega_cut_in,omega_max,21)*np.pi/30 # rad/s
Q = (0.5*rho*omega_array**2*R**3 * A * max_cp ) / tsr_opt**3 
Paero_array = Q*omega_array
if Paero_array[-1] < rated_power_aero: # then we have region 2and1/2
    Region2andAhalf = True
    Q_extra = rated_power_aero/(omega_max*np.pi/30)
    Q = np.append(Q,Q_extra)
    u_r2_end = (Paero_array[-1]/(0.5*rho*np.pi*R**2*max_cp))**(1/3);
    omega_array = np.append(omega_array,omega_array[-1])
    Paero_array = np.append(Paero_array,rated_power_aero)
else: # limit aero_power to the last Q*omega_max
    rated_power_aero = Paero_array[-1]
u_rated = (rated_power_aero/(0.5*rho*np.pi*R**2*max_cp))**(1/3);
u_array = np.linspace(2,25,46)
idx = np.argmin(np.abs(u_array-u_rated))
if u_rated > u_array[idx]:
    u_array = np.insert(u_array,idx+1,u_rated)
else:
    u_array = np.insert(u_array,idx,u_rated)
## Solve aero-mechanical power balance for yaw angles in array gamma_arr
c = 0
gamma_arr = np.linspace(-30,30,161)
cp_g1 = np.zeros_like(gamma_arr)
ct_g1 = np.zeros_like(gamma_arr)
pitch_angle = np.zeros_like(gamma_arr)
tip_speed_ratio = np.zeros_like(gamma_arr)
region3 = np.zeros_like(gamma_arr)
for gamma in gamma_arr:    
    # load control trajectories (i.e. omega rated and torque(omega))
    pow_lut_omega = Paero_array;
    omega_lut_pow = omega_array*30/np.pi;
    torque_lut_omega = Q;
    omega_lut_torque = omega_lut_pow;
    omega_rated = omega_max*np.pi/30; #rad/s
    if u_v > u_rated:
        tsr_v = omega_rated*R/u_v*np.cos(np.deg2rad(gamma))**0.5;
    else:
        tsr_v = tsr_opt*np.cos(np.deg2rad(gamma))**0.5;
    if Region2andAhalf: # fix for interpolation
        omega_lut_torque[-1] = omega_lut_torque[-1]+1e-2;
        omega_lut_pow[-1]    = omega_lut_pow[-1]+1e-2;
    data = gamma,tilt,u_v,pitch_opt,omega_lut_pow,torque_lut_omega,shear,rho,cd,cl_alfa,beta
    [tsr_out_soluzione,infodict,ier,mesg] = fsolve(get_tsr,tsr_v,args=data,full_output=True)
    # check if solution was possible. If not, we are in region 3
    if (np.abs(infodict['fvec']) > 10 or tsr_out_soluzione < 4):
        tsr_out_soluzione = 1000;
    # save solution
    tsr_outO = tsr_out_soluzione;
    omega    = tsr_outO*u_v/R;
    # check if we are in region 2 or 3 
    if omega < omega_rated: # region 2
        pitch_out0 = pitch_opt; # Define optimum pitch
    else: # region 3
        region3[c] = 1
        tsr_outO = omega_rated*R/u_v;
        data = gamma,tilt,u_v,omega_rated,omega_array,Q,shear,rho,cd,cl_alfa,beta
        # solve aero-electrical power balance with TSR from rated omega
        [pitch_out_soluzione,infodict,ier,mesg] = fsolve(get_pitch,10,args=data,factor=0.1,full_output=True)    
        if pitch_out_soluzione < pitch_opt:
            print('problem w/ PITCH')
            pitch_out_soluzione = pitch_opt
        pitch_out0 = pitch_out_soluzione;
    
    # COMPUTE CP AND CT GIVEN THE PITCH AND TSR FOUND ABOVE
    passo         = pitch_out0
    tipspeedratio = tsr_outO
    # Cp and Ct in MISALIGNED conditions (gamma != 0)
    mu    = np.arccos(np.cos(np.deg2rad(gamma))*np.cos(np.deg2rad(tilt)))
    data  = (sigma,cd,cl_alfa,gamma,tilt,shear,np.cos(mu),np.sin(mu),tsr_outO,np.deg2rad(passo)+np.deg2rad(beta),mu)
    x0    = 0.8
    ct = fsolve(find_ct, x0,args=data)[0]
    cp = find_cp(sigma,cd,cl_alfa,gamma,tilt,shear,np.cos(mu),np.sin(mu),tsr_outO,np.deg2rad(passo)+np.deg2rad(beta),ct,mu)
    # Cp and Ct in ALIGNED conditions (gamma = 0)
    mu    = np.arccos(np.cos(np.deg2rad(0))*np.cos(np.deg2rad(tilt)))
    data  = (sigma,cd,cl_alfa,0,tilt,shear,np.cos(mu),np.sin(mu),tsr_outO,np.deg2rad(passo)+np.deg2rad(beta),mu)
    x0    = 0.8
    ct0 = fsolve(find_ct, x0,args=data)[0]
    cp0 = find_cp(sigma,cd,cl_alfa,0,tilt,shear,np.cos(mu),np.sin(mu),tsr_outO,np.deg2rad(passo)+np.deg2rad(beta),ct0,mu)
    # compute power/thrust loss factors
    eta_p = cp/cp0;
    eta_t = ct/ct0;
    # get cp from LUT and apply power loss factor
    interp   = RegularGridInterpolator((np.squeeze((u_table_iea)),np.squeeze((tsr_table_iea)),
                        np.squeeze((pitch_table_iea))), cp_table_iea,
                                       bounds_error=False, fill_value=None)
    Cp_now = interp((9,tipspeedratio,passo),method='linear');
    cp_g1[c] =  Cp_now*eta_p;
    # get ct from LUT and apply power loss factor
    interp   = RegularGridInterpolator((np.squeeze((u_table_iea)),np.squeeze((tsr_table_iea)),
                        np.squeeze((pitch_table_iea))), ct_table_iea,
                                       bounds_error=False, fill_value=None)
    Ct_now = interp((9,tipspeedratio,passo),method='linear');
    ct_g1[c] =  Ct_now*eta_t;
    pitch_angle[c] = passo
    tip_speed_ratio[c] = tipspeedratio        
    c+=1
    
## Determine shading areas for plot, i.e. conditions above rated
idx = np.squeeze(np.where(region3==1))
y_fill = np.array([-100,100])
try:
    x1 = np.array([gamma_arr[idx[0]],gamma_arr[idx[0]]])
    x2 = np.array([gamma_arr[idx[-1]],gamma_arr[idx[-1]]])
except:
    x1 = np.array([-50,-50])
    x2 = np.array([-40,-40])

 Plot!

In [ ]:
plt.figure(2,figsize=(9,4))
plt.subplot(2,2,1)
plt.plot(gamma_arr,ct_g1)
plt.ylabel('$C_T$ [-]')
plt.fill_betweenx(y_fill,x1,x2,alpha=0.2,color='C1',zorder=-100)
# plt.xticks(np.linspace(-30,30,7),color='w')
plt.xlim([gamma_arr[0],gamma_arr[-1]])
# plt.text(24,0.75,'(a)')
plt.ylim([np.floor(np.min(ct_g1*10))/10,np.ceil(np.max(ct_g1*10))/10])
# plt.yticks([0.6,0.7,0.8])
plt.grid()
plt.subplot(2,2,2)
plt.plot(gamma_arr,cp_g1, label='_nolegend_')
plt.ylabel('$C_P$ [-]')
plt.grid()
plt.ylim([np.floor(np.min(cp_g1*10))/10,np.ceil(np.max(cp_g1*10))/10])
# plt.yticks(np.linspace(0.35,0.5,4))
# plt.xticks(np.linspace(-30,30,7),color='w')
plt.fill_betweenx(y_fill,x1,x2,alpha=0.2,color='C1',label='Above rated',zorder=-100)
# plt.xticks(np.linspace(-30,30,7))
plt.xlim([gamma_arr[0],gamma_arr[-1]])
plt.subplot(2,2,3)
plt.plot(gamma_arr,pitch_angle)
plt.grid()
plt.xlabel('$\gamma$ [$^\circ$]')
plt.ylabel(r'$\theta_p$ [$^\circ$]')
plt.fill_betweenx(y_fill,x1,x2,alpha=0.2,color='C1',zorder=-100)
plt.ylim([0,np.ceil(np.max(pitch_angle*10))/10+1])
# plt.xticks(np.linspace(-30,30,7))
plt.xlim([gamma_arr[0],gamma_arr[-1]])
# plt.ylim([0,4])
plt.subplot(2,2,4)
plt.plot(gamma_arr,tip_speed_ratio)
plt.grid()
plt.xlabel('$\gamma$ [$^\circ$]')
plt.ylabel('$\lambda$ [-]')
plt.fill_betweenx(y_fill,x1,x2,alpha=0.2,color='C1',label='Above rated',zorder=-100)
plt.legend(ncol=1,loc='lower left',bbox_to_anchor=(0.15,0),frameon=True)
# plt.xticks(np.linspace(-30,30,7))
plt.xlim([gamma_arr[0],gamma_arr[-1]])
plt.ylim([np.floor(np.min(tip_speed_ratio*10))/10,np.ceil(np.max(tip_speed_ratio*10))/10])
# plt.yticks(np.linspace(7,7.8,3))
plt.subplots_adjust(left=0.1, bottom=0.2, right=0.95, top=0.95, wspace=0.3, hspace=0.2)